### 회기동 음식점 리스트 상세페이지 크롤링

- 수집 목록: 식당명, 주소, 메뉴
- 음식점 리스트: 2020년 9월 기준 동대문구 음식점 공공데이터 사용

* 네이버...크롤링 다 막아놔서...식당 리스트 공공데이터 말고 지도에 있는 걸루 가져오고 싶었지만...어쩔 수 없었당...
* 식당들마다 이름이 아니라 자기네들이 지정한 고유키로 url을 구성해 놓아서 키도 하나씩 찾아줘야 했는데... BeautifulSoup으로 긁히지도 않아서... 셀레니움으로 긁다가... 100번 정도 긁으면 나한테 404 에러 주고... 그래서 나눠 긁고... 내가 네이버 정보 가져가봤자... 얼마나 가져간다고... 증말 너무하네...

In [68]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import datetime
import time
import os
from requests import get
import pandas as pd
import json

In [2]:
#공공데이터에서 회기동 음식점 리스트 얻기
location = "회기"
res_info = pd.read_csv("./dongdaemoon.csv", engine = "python", encoding = "CP949")
res_info = res_info.fillna("")

hoegi = res_info[res_info["소재지(지번)"].str.contains(location)]
rest_name = hoegi["업소명"]
rest_name.head(10)

4          일미기사식당
12     홍콩반점(경희대점)
61           노랑통닭
65       노바 이탈리아노
94     송가네 오소리순대국
102             설
111       원조할머니보쌈
156         레프트커피
179           미식성
180           삼성루
Name: 업소명, dtype: object

In [8]:
#음식점 별 네이버 data-cid 추출하기
id_list = []

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.headless = True
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15")
options.add_argument("lang=ko_KR")

def crawlId(rest_name, id_list):
    driver = webdriver.Chrome("./chromedriver")
    driver.implicitly_wait(3)

    for rest in rest_name:
        search_url = "https://m.map.naver.com/search2/search.naver?query=" + location + " " + rest + "&sm=hty&style=v5"
        driver.get(search_url)
        time.sleep(2)

        try:
            link = driver.find_element_by_css_selector("a.a_item")
            id_list.append(link.get_attribute("data-cid"))
        except:
            print(rest + " 패스")

In [10]:
crawlId(rest_name[0:100], id_list)

콩불 경희대점 패스
안씨네원조쭈꾸미 패스
씽씽스넥 패스
깡통집 패스
에피소드 패스
옥상아래 꽃집 패스
서병장대김이병 패스
새마당(황칠염소탕.영양탕.황칠삼계탕) 패스
세겹살닷컴(.COM) 패스
오바(OBAR) 패스
용용이네떡볶이 패스
포보허수아비 패스
한탄강메기의추억 패스
피지(PG) 패스
솔레미오 패스
아라마크(주)경희의료원 장례식장 패스
아쿠아플러스 패스


In [35]:
crawlId(rest_name[100:200], id_list)

8번지 패스
황하 패스
오리마을 패스
미스터피자(경희대점) 패스
만원의행복 패스
또순이순대보쌈 패스
기리반점 패스
오아시스(O:asis) 패스
더 스토리 패스
티엘씨(TLC) 패스
시광리 패스
(주)신년에프앤비 패스


In [43]:
crawlId(rest_name[300:400], id_list)

세븐틴(17) 패스
이상한나라의앨리스 패스
찬쥐가 패스
부어케그리고까를로봉봉 패스
비스테어·피자나폴리 패스
콕찍어콕콕꼬치다 패스
서선생(先生)묵은지 패스
버거디 패스
MJ24불가마스파랜드 패스
사라다방 패스
일폐이중국전통요리 패스
포기(foggy) 패스
모스플레이스(MODSPLACE) 패스
카도오 패스
청춘텐트포차 패스
또뽀끼야 경희대점 패스
신(辛)장군마라탕 패스
꽃담찬 패스
커버(cover)21 패스
정앤정 JNJ 패스
세성당 패스
오늘도무사히 패스
아(A)만두여기요 패스
다향 패스
더 와요(The wayo) 패스
파파이스경희의료원점 패스
커피순(Coffee SOON) 패스
디초콜릿커피앤드경희대점 패스
커피숲 패스


In [55]:
crawlId(rest_name[400:], id_list)

죽이야기 경희의료원점 패스
커피온리경희대점 패스
아비앙또(Abiantot) 패스
미소당본점 패스
세종분식 패스
투고샐러드 경희대학교점 패스
카페쿠피 패스
쥬스앤아이 패스
주식회사로봇에프앤비로봇김밥경희대점 패스
빙달회기점 패스
시즌아이피시방 패스
카페쿠피 패스
푸른솔 문화관 카페 패스
케이원커피 패스
못난이 꽈배기(회기역게이트안제과점) 패스
한의대카페 패스
(주)이디야경희대캠퍼스점 패스
1일1디저트 패스
(주)커피빈코리아경희대점 패스
마을활력소회기한지붕 패스
홍차오 패스
청운관 샐러드점 패스
더나인피씨방 패스
공유주방 썬플라워(SUNFLOWER) 패스
라이또PC경희대점 패스
과일차(7+0) 패스
메가엠지씨커피 경희대점 패스
디브이카페(DV CAFE) 패스


#### 상세 페이지 크롤링

In [71]:
names, addresses, menu_lists = [], [], []

In [72]:
for id_ in id_list:
    
    #음식점 메인 페이지 request
    url = "https://m.place.naver.com/restaurant/" + id_ + "/home"
    headers={"User-Agent": "Mozilla/5.0"}
    main_req = get(url)
    soup = BeautifulSoup(main_req.content.decode('utf-8','replace'))
    time.sleep(5)
    #음식점 이름, 주소 크롤링
    try:
        place_name = soup.select_one("span._3XamX").get_text()
        address = soup.select_one("span._2yqUQ").get_text()

        #메뉴 리스트 페이지 request
        menu_url = url.replace("/home", "") + "/menu/list"
        menu_req = get(menu_url)
        menu_soup = BeautifulSoup(menu_req.content.decode("utf-8", "replace"))

        #메뉴 리스트 크롤링
        menu_list = menu_soup.select("span._3yfZ1")
        for (idx, menu) in enumerate(menu_list):
            menu_list[idx] = menu.get_text()

        #리스트 저장
        names.append(place_name)
        addresses.append(address)
        menu_lists.append(menu_list)
    
    except:
        print(url)

In [73]:
#데이터 프레임으로 저장
df = pd.DataFrame()
df["식당명"] = names
df["주소"] = addresses
df["메뉴"] = menu_lists

df

,식당명,주소,메뉴
0,일미기사식당,서울 동대문구 회기로 105,[]
1,홍콩반점0410 경희대점,서울 동대문구 경희대로 12-1,"[짬뽕, 짜장면, 탕수육(소), 짬뽕밥, 짜장밥, 군만두, 쟁반짜장]"
2,노랑통닭 경희대점,서울 동대문구 경희대로1길 13 1층,"[알싸한 마늘치킨 레귤러, 알싸한 마늘치킨 사이즈업, 엄청 큰 후라이드 치킨, 엄청..."
3,노바 이탈리아노,서울 동대문구 회기로21길 44,"[안심,연어,닭고기 스테이크, 샐러드, 파스타, 피자, 하우스와인]"
4,송가네오소리순대국,서울 동대문구 회기로13길 10,"[술국, 곱창전골(소), 순대국, 오소리반반, 순대철판곱창]"
...,...,...,...
352,폴링인풀 포켓클럽,"서울 동대문구 회기로13길 6 3층(회기동, 이설빌딩)","[정액제(3시간) 인당, 1시간(2인)당, 1시간(3인)당, 1시간(4인이상)당]"
353,이츠이츠 경희대점,서울 동대문구 회기로21길 25 2층,"[코코넛스무디커피, 코코넛쉐이크, (ICE) 코코넛 카푸치노, (HOT) 코코넛 카..."
354,메모아르,서울 동대문구 회기로 156 2층,"[아메리카노, 카페라떼, 당근케이크, 아몬드초코, 딸기레어치즈케이크, 티라미슈]"
355,카페 마타타 경희대점,서울 동대문구 회기로21길 44,[]


In [74]:
df.to_csv('회기동 음식점 리스트.csv', index = False)

#### 메뉴리스트 정리

In [85]:
df_valid = df[df["메뉴"].apply(lambda x: len(x)) != 0]

In [87]:
df_valid

,식당명,주소,메뉴
1,홍콩반점0410 경희대점,서울 동대문구 경희대로 12-1,"[짬뽕, 짜장면, 탕수육(소), 짬뽕밥, 짜장밥, 군만두, 쟁반짜장]"
2,노랑통닭 경희대점,서울 동대문구 경희대로1길 13 1층,"[알싸한 마늘치킨 레귤러, 알싸한 마늘치킨 사이즈업, 엄청 큰 후라이드 치킨, 엄청..."
3,노바 이탈리아노,서울 동대문구 회기로21길 44,"[안심,연어,닭고기 스테이크, 샐러드, 파스타, 피자, 하우스와인]"
4,송가네오소리순대국,서울 동대문구 회기로13길 10,"[술국, 곱창전골(소), 순대국, 오소리반반, 순대철판곱창]"
5,설,서울 동대문구 경희대로 9,"[회덮밥, 오뎅백반, 고기덮밥, 우동, 유부초밥]"
...,...,...,...
350,Cafe H,서울 동대문구 회기로23가길 53,[아메리카노]
352,폴링인풀 포켓클럽,"서울 동대문구 회기로13길 6 3층(회기동, 이설빌딩)","[정액제(3시간) 인당, 1시간(2인)당, 1시간(3인)당, 1시간(4인이상)당]"
353,이츠이츠 경희대점,서울 동대문구 회기로21길 25 2층,"[코코넛스무디커피, 코코넛쉐이크, (ICE) 코코넛 카푸치노, (HOT) 코코넛 카..."
354,메모아르,서울 동대문구 회기로 156 2층,"[아메리카노, 카페라떼, 당근케이크, 아몬드초코, 딸기레어치즈케이크, 티라미슈]"


In [91]:
total_menu = []

for menu in df["메뉴"]:
    total_menu += menu

print(total_menu)

['짬뽕', '짜장면', '탕수육(소)', '짬뽕밥', '짜장밥', '군만두', '쟁반짜장', '알싸한 마늘치킨 레귤러', '알싸한 마늘치킨 사이즈업', '엄청 큰 후라이드 치킨', '엄청 큰 깐풍치킨', '순살 3종 세트', '엄청 큰 양념치킨', '매콤 후라이드', '엄청 큰 반반치킨 (후라이드 + 양념)', '엄청 큰 반반치킨 (후라이드 + 깐풍)', '엄청 큰 반반치킨 (양념 + 깐풍)', '고바치', '간바치', '웰빙 파닭 (후라이드 + 양념)', '웰빙 파닭 (후라이드 + 깐풍)', '웰빙 파닭 (양념 + 깐풍)', '안심,연어,닭고기 스테이크', '샐러드', '파스타', '피자', '하우스와인', '술국', '곱창전골(소)', '순대국', '오소리반반', '순대철판곱창', '회덮밥', '오뎅백반', '고기덮밥', '우동', '유부초밥', '아보카도 브런치', '브라운치즈 크로플', '독일식 팬케이크 플레이트', '크래미 샌드위치', '얼그레이라떼', '딥카라멜라떼', '아메리카노', '짜장면', '탕수육+짜장1', '탕수육+짬뽕1', '탕수육+볶음밥1', '탕수육+쟁반짜장2인+만두or콜라', '탕볶밥', '볶짜면', '탕짬면', '사천탕수육', '닭탕수육', '찹쌀탕수육 大', '버섯매운탕칼국수', '바지락칼국수', '시골콩국수', '소고기 샤브샤브', '냉면', '기본카레라이스', '가지카레라이스', '야채카레라이스', '버섯카레라이스', '허브치킨카레라이스', '포크카레라이스', '함바그카레라이스', '비프카레라이스', '알새우카레라이스', '해산물카레라이스', '기본카레우동', '버섯카레우동', '허브치킨카레우동', '비프카레우동', '알새우카레우동', '족발 특대', '보쌈 특대', '족발 대', '보쌈 대', '족발 중', '보쌈 중', '족발 소', '보쌈 소', '쟁반막국수 대', '쟁반막국수 중', '춘천막국수', '탕수육+차돌박이짬뽕', '탕수육+속풀이짬뽕', '탕수육+불쟁반1', '사천탕수육+깐풍새우or깐풍육+짜장or간짜장or짬

In [95]:
#중복 제거
menu = list(set(total_menu))

In [100]:
len(menu)

1542

In [97]:
menu

['얼큰해물짬뽕탕',
 '보쌈정식',
 '샹라러우쓰',
 '낙지볶음',
 '무뼈닭발+오돌뼈+계란찜+주먹밥',
 '매운쌀국수',
 '플레인요거트스무디',
 '외에도 20가지 넘는 메뉴가 다양하게 있습니다',
 '바질페스토',
 '삼선짬뽕',
 '불곱창',
 '비엘티',
 '흑마늘족발(앞발 大)',
 '辛뽀그리',
 '코코넛스무디커피',
 '풍기',
 '벌집껍데기 160g',
 '탕볶밥',
 '닭발(무뼈)주먹밥',
 '순대볶음',
 '복숭아꽃술750ml',
 '고기통만두8개',
 '삐뚤이소라 초무침',
 '라지 사이즈 커피',
 '바닐라라떼',
 '수족한상차림',
 '1인당',
 '광어 중',
 '버터꿀봉-매콤땡초',
 'ICE / HOT 허니몽',
 '김치통만두8개',
 '목화 라떼 초콜릿, 딸기',
 '군만두',
 '숙주라면',
 '뼈찜 중(3~4인분)',
 '국내산 돼지 항정살 180g',
 '우럭',
 '소고기 육전',
 '1인사시미',
 '머랭케이크',
 '마라롱샤',
 '고로케',
 '분짜(Bun Cha)',
 '안창살',
 '뿌링꿀호떡 ',
 '연태고량(대)',
 '콩나물굴국',
 '튤립닭발(국물)1인분',
 '슬라이스',
 '물냉면',
 '항정살 1인분',
 '칠성마약찜닭',
 '밀푀유 치즈돈가스',
 '차돌된장찌개비빔밥',
 '능이반계탕',
 '안동소주, 문배주, 이강주, 소주 등',
 '생연어샐러드',
 '오겹살 120g',
 '국내산 한우 1+ 꽃등심 180g',
 '밀푀유소고기나베',
 '생목살',
 '김치말이냉국수',
 '칼국수',
 '빠네크림파스타',
 '허니 마늘족발(앞발 大)',
 '전어',
 '고추맵봉-달콤치즈',
 '볶음면',
 '지엔빙,냉면구이',
 '쿠웨이트떡볶이',
 '갈매기살',
 '똥집후라이드',
 '가리비구이',
 '수제비/칼제비',
 '계란볶음밥',
 '아보카도 브런치',
 '버터구이오징어',
 '로주탄(중)',
 '음료(소)',
 '오렌지주스',
 '하우스와인 1잔',
 '앙게트',
 '아메리카노 L',
 '시래기황태

In [101]:
#한 사람당 수집해야 하는 개수
len(menu) / 6

257.0

In [121]:
#파일로 저장하기
f = open("./회기동 메뉴리스트.txt", 'w')
for (idx, m) in enumerate(menu):
    if (idx % int(len(menu) / 6) == 0):
        f.write("=" * 40 + " " + "(" + str(int(idx / int(len(menu) / 6))) + ")" + "\n")
    f.write(m + '\n')
f.close()